In [1]:
from pyspark.sql.types import (
   StructType, StructField, StringType, IntegerType, TimestampType
)

In [2]:
schema = StructType([
    StructField('crime_id', StringType(), True),
    StructField('original_crime_type_name', StringType(), True),
    StructField('report_date', TimestampType(), True),
    StructField('call_date', TimestampType(), True),
    StructField('offense_date', TimestampType(), True),
    StructField('call_time', StringType(), True),
    StructField('call_date_time', TimestampType(), True),
    StructField('disposition', StringType(), True),
    StructField('address', StringType(), True),
    StructField('city', StringType(), True),
    StructField('state', StringType(), True),
    StructField('agency_id', StringType(), True),
    StructField('address_type', StringType(), True),
    StructField('common_location', StringType(), True)
])

In [3]:
df = spark.read.option("multiline","true") \
    .json('data/police-department-calls-for-service.json', schema=schema)

In [4]:
df.printSchema()

root
 |-- crime_id: string (nullable = true)
 |-- original_crime_type_name: string (nullable = true)
 |-- report_date: timestamp (nullable = true)
 |-- call_date: timestamp (nullable = true)
 |-- offense_date: timestamp (nullable = true)
 |-- call_time: string (nullable = true)
 |-- call_date_time: timestamp (nullable = true)
 |-- disposition: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- agency_id: string (nullable = true)
 |-- address_type: string (nullable = true)
 |-- common_location: string (nullable = true)



In [5]:
df.limit(5).toPandas()

,crime_id,original_crime_type_name,report_date,call_date,offense_date,call_time,call_date_time,disposition,address,city,state,agency_id,address_type,common_location
0,183653763,Traffic Stop,2018-12-31,2018-12-31,2018-12-31,23:57,2018-12-31 23:57:00,ADM,Geary Bl/divisadero St,San Francisco,CA,1,Intersection,
1,183653756,Traf Violation Cite,2018-12-31,2018-12-31,2018-12-31,23:54,2018-12-31 23:54:00,CIT,100 Blk Howard St,San Francisco,CA,1,Geo-Override,
2,183653746,Passing Call,2018-12-31,2018-12-31,2018-12-31,23:49,2018-12-31 23:49:00,HAN,3300 Block Of 20th Av,San Francisco,CA,1,Common Location,"Stonestown Galleria, Sf"
3,183653745,Audible Alarm,2018-12-31,2018-12-31,2018-12-31,23:47,2018-12-31 23:47:00,PAS,1900 Block Of 18th Av,San Francisco,CA,1,Premise Address,
4,183653737,Traffic Stop,2018-12-31,2018-12-31,2018-12-31,23:46,2018-12-31 23:46:00,CIT,Sansome St/chestnut St,San Francisco,CA,1,Intersection,


In [6]:
distinct_table = df.select(['original_crime_type_name', 'disposition'])

In [27]:
agg_df = distinct_table.groupBy('original_crime_type_name').count()
agg_df = agg_df.orderBy(agg_df['count'].desc())

In [28]:
agg_df.show()

+------------------------+-----+
|original_crime_type_name|count|
+------------------------+-----+
|            Passing Call|35180|
|            Traffic Stop|13538|
|     Traf Violation Cite|10921|
|       Suspicious Person|10007|
|      Homeless Complaint| 6072|
|              Trespasser| 5840|
|           Audible Alarm| 5775|
|                  22500e| 5651|
|        Well Being Check| 5552|
|         Muni Inspection| 5509|
|      Suspicious Vehicle| 5031|
|         Fight No Weapon| 4199|
|          Noise Nuisance| 4072|
|      Auto Boost / Strip| 2969|
|      Traf Violation Tow| 2915|
|                    Poss| 2851|
|      Mentally Disturbed| 2585|
|             Petty Theft| 2549|
|          Meet W/citizen| 2414|
|       Assault / Battery| 2356|
+------------------------+-----+
only showing top 20 rows



In [29]:
radio_code_json_filepath = "data/radio_code.json"
radio_code_df = spark.read \
    .option("multiline","true") \
    .json(radio_code_json_filepath)
radio_code_df = radio_code_df.withColumnRenamed("disposition_code", "disposition")

In [30]:
radio_code_df.toPandas()

,description,disposition
0,Abated,ABA
1,Admonished,ADM
2,Advised,ADV
3,Arrest,ARR
4,Cancel,CAN
5,CPSA assignment,CSA
6,Cancel,22
7,Cited,CIT
8,Criminal Activation,CRM
9,Gone on Arrival,GOA
